In [9]:
import numpy as np
import pandas as pd
from collections import defaultdict
from joblib import Parallel, delayed
import geopandas as gpd

In [2]:
import os 
import sys 
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from StreamCat_functions_gpu import bastards, dbf2DF, nhd_dict, make_all_cat_comids

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [11]:
#TODO check PlusFlow.dbf and NHDFlowline.dbf for geometry values
flow_path = "O:/PRIV/CPHEA/PESD/COR/CORFILES/Geospatial_Library_Resource/Physical/HYDROLOGY/NHDPlusV21/NHDPlusMS/NHDPlus06/NHDPlusAttributes/PlusFlow.dbf"
flow = gpd.read_file(flow_path)[["TOCOMID", "FROMCOMID"]] # dbf2DF(flow_path)
flow

,TOCOMID,FROMCOMID
0,19744346,19744328
1,19744330,0
2,19744346,19744330
3,19745126,19744344
4,19744344,19744314
...,...,...
83470,14640911,14640899
83471,14640855,0
83472,14640903,14640855
83473,14640853,0


In [12]:
fls_path = "O:/PRIV/CPHEA/PESD/COR/CORFILES/Geospatial_Library_Resource/Physical/HYDROLOGY/NHDPlusV21/NHDPlusMS/NHDPlus06/NHDSnapshot/Hydrography/NHDFlowline.dbf"
# could just do gpd.read_file(fls_path)
fls = gpd.read_file(fls_path) # dbf2DF(fls_path)
fls

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\pyogrio\raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(


,COMID,FDATE,RESOLUTION,GNIS_ID,GNIS_NAME,LENGTHKM,REACHCODE,FLOWDIR,WBAREACOMI,FTYPE,FCODE,SHAPE_LENG,ENABLED,GNIS_NBR,geometry
0,19763248,1999-06-17,Medium,1487063,North Fork Holston River,0.017,06010101000001,With Digitized,-9998,ArtificialPath,55800,0.000181,True,0,"LINESTRING Z (-82.61375 36.55994 0, -82.61368 ..."
1,19762966,1999-06-17,Medium,1487063,North Fork Holston River,1.155,06010101000001,With Digitized,0,StreamRiver,46006,0.011100,True,0,"LINESTRING Z (-82.61713 36.56864 0, -82.61695 ..."
2,19763250,1999-06-17,Medium,1487063,North Fork Holston River,1.487,06010101000001,With Digitized,120049405,ArtificialPath,55800,0.013993,True,0,"LINESTRING Z (-82.61377 36.55985 0, -82.61521 ..."
3,19762962,1999-06-17,Medium,1487063,North Fork Holston River,1.244,06010101000002,With Digitized,0,StreamRiver,46006,0.011197,True,0,"LINESTRING Z (-82.6163 36.57959 0, -82.61656 3..."
4,19762950,1999-06-17,Medium,1487063,North Fork Holston River,0.240,06010101000003,With Digitized,0,StreamRiver,46006,0.002670,True,0,"LINESTRING Z (-82.61369 36.57938 0, -82.61463 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59352,936040017,2009-03-15,Medium,517033,Tennessee River,0.348,06040006008211,With Digitized,120049866,ArtificialPath,55800,0.014260,True,0,"LINESTRING Z (-88.31777 37.0525 0, -88.31842 3..."
59353,19696135,2009-04-02,Medium,1292451,Mackey Branch,0.739,06020001002394,With Digitized,0,StreamRiver,46006,0.002356,True,0,"LINESTRING Z (-85.17101 35.00158 0, -85.17234 ..."
59354,19674709,1999-09-07,Medium,None,None,2.895,06020002013122,With Digitized,0,StreamRiver,46006,0.023544,True,0,"LINESTRING Z (-83.97752 35.00462 0, -83.97721 ..."
59355,19647060,1999-11-23,Medium,None,None,5.319,06030001013038,With Digitized,0,StreamRiver,46006,0.050781,True,0,"LINESTRING Z (-86.01452 34.83018 0, -86.01433 ..."


In [13]:
inter_tbl = pd.read_csv("../config_tables/InterVPU.csv")

In [ ]:
all_comids = np.load('accum_npy/allCatCOMs.npz')
all_comids

In [ ]:
pre = "O:/PRIV/CPHEA/PESD/COR/CORFILES/Geospatial_Library_Resource/Physical/HYDROLOGY/NHDPlusV21/NHDPlusMS/NHDPlus06"
zone = '06'

In [16]:
coastfl = fls.COMID[fls.FTYPE == "Coastline"]
flow = flow[~flow.FROMCOMID.isin(coastfl.values)]
flow = flow[~flow.FROMCOMID.isin(inter_tbl.removeCOMs)]
out = np.setdiff1d(flow.FROMCOMID.values, fls.COMID.values)
out = out[np.nonzero(out)]
flow = flow[~flow.FROMCOMID.isin(np.setdiff1d(out, inter_tbl.thruCOMIDs.values))]

flow_dict = defaultdict(list)
for _, row in flow.iterrows():
    flow_dict[row.TOCOMID].append(row.FROMCOMID)

for interLine in inter_tbl.values:
    if interLine[6] > 0 and interLine[2] == zone:
        flow_dict[int(interLine[6])].append(int(interLine[0]))

out_of_vpus = inter_tbl.loc[
    (inter_tbl.ToZone == zone) & (inter_tbl.DropCOMID == 0)
].thruCOMIDs.values
cats = dbf2DF(f"{pre}/NHDPlusCatchment/Catchment.dbf").set_index("FEATUREID")
comids = cats.index.values
comids = np.append(comids, out_of_vpus)

ups = [list(all_comids.intersection(bastards(x, flow_dict))) for x in comids]
lengths = np.array([len(u) for u in ups])
upstream = np.hstack(ups).astype(np.int32)

AttributeError: 'NpzFile' object has no attribute 'intersection'

In [ ]:
assert len(ups) == len(lengths) == len(comids)
np.savez_compressed(
    f"./accum_npy/accum_{zone}_speed_test.npz",
    comids=comids,
    lengths=lengths,
    upstream=upstream,
)

In [3]:
def process_zone(zone, hr, nhd, inter_tbl, all_comids):
    #print(zone, end=", ", flush=True)
    pre = f"{nhd}/NHDPlus{hr}/NHDPlus{zone}"
    flow = dbf2DF(f"{pre}/NHDPlusAttributes/PlusFlow.dbf")[["TOCOMID", "FROMCOMID"]]
    flow = flow[(flow.TOCOMID != 0) & (flow.FROMCOMID != 0)]
    fls = dbf2DF(f"{pre}/NHDSnapshot/Hydrography/NHDFlowline.dbf")
    coastfl = fls.COMID[fls.FTYPE == "Coastline"]
    flow = flow[~flow.FROMCOMID.isin(coastfl.values)]
    flow = flow[~flow.FROMCOMID.isin(inter_tbl.removeCOMs)]
    out = np.setdiff1d(flow.FROMCOMID.values, fls.COMID.values)
    out = out[np.nonzero(out)]
    flow = flow[~flow.FROMCOMID.isin(np.setdiff1d(out, inter_tbl.thruCOMIDs.values))]
    
    flow_dict = defaultdict(list)
    for _, row in flow.iterrows():
        flow_dict[row.TOCOMID].append(row.FROMCOMID)
    
    for interLine in inter_tbl.values:
        if interLine[6] > 0 and interLine[2] == zone:
            flow_dict[int(interLine[6])].append(int(interLine[0]))
    
    out_of_vpus = inter_tbl.loc[
        (inter_tbl.ToZone == zone) & (inter_tbl.DropCOMID == 0)
    ].thruCOMIDs.values
    cats = dbf2DF(f"{pre}/NHDPlusCatchment/Catchment.dbf").set_index("FEATUREID")
    comids = cats.index.values
    comids = np.append(comids, out_of_vpus)
    
    ups = [list(all_comids.intersection(bastards(x, flow_dict))) for x in comids]
    lengths = np.array([len(u) for u in ups])
    upstream = np.hstack(ups).astype(np.int32)
    
    assert len(ups) == len(lengths) == len(comids)
    np.savez_compressed(
        f"./accum_npy/accum_{zone}_speed_test.npz",
        comids=comids,
        lengths=lengths,
        upstream=upstream,
    )

In [ ]:
def makeNumpyVectors(inter_tbl, nhd):
    os.makedirs("accum_npy", exist_ok=True)
    inputs = nhd_dict(nhd)
    del inputs['16']
    del inputs['15']
    print(inputs)
    all_comids = np.load("accum_npy/allCatComs.npz") #make_all_cat_comids(nhd, inputs) #TODO this function take ~80 minutes
    print("Making numpy files in zone...", end="", flush=True)

    # Parallel processing
    print("Begining parallel execution:")
    Parallel(n_jobs=-1)(delayed(process_zone)(zone, hr, nhd, inter_tbl, all_comids) for zone, hr in inputs.items())


In [ ]:
# TESTING
# could use dask arrays instead of writing to files
# add to above function
import dask.array as da
def makeNumpyVectors_dask(inter_tbl, nhd):
    os.makedirs("accum_npy", exist_ok=True)
    inputs = nhd_dict(nhd)
    inputs.pop('16')
    print(inputs)
    all_comids = make_all_cat_comids(nhd, inputs)
    print("Making numpy files in zone...", end="", flush=True)

    # Initialize empty lists to store results
    comids_list = []
    lengths_list = []
    upstream_list = []

    def process_zone(zone, hr, nhd, inter_tbl, all_comids):
        print(zone, end=", ", flush=True)
        pre = f"{nhd}/NHDPlus{hr}/NHDPlus{zone}"
        flow = dbf2DF(f"{pre}/NHDPlusAttributes/PlusFlow.dbf")[["TOCOMID", "FROMCOMID"]]
        flow = flow[(flow.TOCOMID != 0) & (flow.FROMCOMID != 0)]
        fls = dbf2DF(f"{pre}/NHDSnapshot/Hydrography/NHDFlowline.dbf")
        coastfl = fls.COMID[fls.FTYPE == "Coastline"]
        flow = flow[~flow.FROMCOMID.isin(coastfl.values)]
        flow = flow[~flow.FROMCOMID.isin(inter_tbl.removeCOMs)]
        
        out = np.setdiff1d(np.array(flow.FROMCOMID.values), np.array(fls.COMID.values))
        out = out[np.nonzero(out)]
        flow = flow[~flow.FROMCOMID.isin(np.asnumpy(np.setdiff1d(out, np.array(inter_tbl.thruCOMIDs.values))))]

        flow_dict = defaultdict(list)
        for _, row in flow.iterrows():
            flow_dict[row.TOCOMID].append(row.FROMCOMID)
        
        for interLine in inter_tbl.values:
            if interLine[6] > 0 and interLine[2] == zone:
                flow_dict[int(interLine[6])].append(int(interLine[0]))
        
        out_of_vpus = inter_tbl.loc[
            (inter_tbl.ToZone == zone) & (inter_tbl.DropCOMID == 0)
        ].thruCOMIDs.values
        cats = dbf2DF(f"{pre}/NHDPlusCatchment/Catchment.dbf").set_index("FEATUREID")
        comids = np.array(cats.index.values)
        comids = np.append(comids, np.array(out_of_vpus))
        
        ups = [list(all_comids.intersection(bastards(x, flow_dict))) for x in np.asnumpy(comids)]
        lengths = np.array([len(u) for u in ups])
        upstream = np.hstack(ups).astype(np.int32)
        
        assert len(ups) == len(lengths) == len(comids)
        
        # Append results to lists
        comids_list.append(np.asnumpy(comids))
        lengths_list.append(np.asnumpy(lengths))
        upstream_list.append(np.asnumpy(upstream))

    # Parallel processing
    Parallel(n_jobs=-1)(delayed(process_zone)(zone, hr, nhd, inter_tbl, all_comids) for zone, hr in inputs.items())

    # Convert lists to Dask arrays
    comids_dask = da.concatenate([da.from_array(arr) for arr in comids_list])
    lengths_dask = da.concatenate([da.from_array(arr) for arr in lengths_list])
    upstream_dask = da.concatenate([da.from_array(arr) for arr in upstream_list])

    # Save Dask arrays to a single file
    da.to_zarr(comids_dask, 'comids.zarr', mode='w')
    da.to_zarr(lengths_dask, 'lengths.zarr', mode='w')
    da.to_zarr(upstream_dask, 'upstream.zarr', mode='w')

In [5]:
inter_vpu = pd.read_csv("../config_tables/InterVPU.csv")
NHD_DIR = "O:/PRIV/CPHEA/PESD/COR/CORFILES/Geospatial_Library_Resource/Physical/HYDROLOGY/NHDPlusV21"
inter_vpu

,thruCOMIDs,FromZone,ToZone,AdjustComs,toCOMIDs,DropCOMID,UpCOMadd,removeCOMs,comments
0,18267741,14,15,0,0,0,20734041,24719331,NaN
1,20734037,14,15,0,0,0,0,10466473,NaN
2,1861888,06,05,0,0,0,0,15714785,NaN
3,1862004,06,05,0,0,0,0,0,NaN
4,1862014,06,05,1862004,0,1862014,0,0,NaN
5,1844789,05,08,0,0,0,0,0,NaN
6,7227390,10U,10L,0,0,0,0,0,NaN
7,11764402,10U,10L,0,0,0,0,0,NaN
8,6018266,10L,07,0,0,0,0,0,NaN
9,5093446,07,08,0,0,0,0,0,NaN


In [6]:
%%timeit
makeNumpyVectors(inter_vpu, NHD_DIR)

OrderedDict({'06': 'MS', '05': 'MS', '10U': 'MS', '10L': 'MS', '07': 'MS', '11': 'MS', '14': 'CO', '01': 'NE', '17': 'PN', '13': 'RG', '12': 'TX', '09': 'SR', '02': 'MA', '08': 'MS', '04': 'GL', '03W': 'SA', '03S': 'SA', '03N': 'SA', '18': 'CA'})
Making allFLOWCOMs numpy file
...done!
Making numpy files in zone...Begining parallel execution:


KeyError: "['geometry'] not found in axis"